In [9]:
pip install konlpy


   ---------------------------------------- 0.0/19.4 MB ? eta -:--:--
   ------- -------------------------------- 3.7/19.4 MB 21.6 MB/s eta 0:00:01
   ---------------- ----------------------- 8.1/19.4 MB 19.9 MB/s eta 0:00:01
   ---------------------------- ----------- 13.6/19.4 MB 22.2 MB/s eta 0:00:01
   ---------------------------------------  19.4/19.4 MB 23.7 MB/s eta 0:00:01
   ---------------------------------------- 19.4/19.4 MB 22.3 MB/s eta 0:00:00

   ---------------------------------------- 0/2 [JPype1]
   -------------------- ------------------- 1/2 [konlpy]
   -------------------- ------------------- 1/2 [konlpy]
   ---------------------------------------- 2/2 [konlpy]



In [9]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options #브라우저의 내부적인 옵션들을 관리하는것 Options
from bs4 import BeautifulSoup
from openpyxl import Workbook # 실제 엑셀파일을 만드는 것 Workbook
from konlpy.tag import Okt # konlpy.tag한글 자연어를 분석해주는 라이브러리 / Okt → 명사만 찾아옴
from collections import Counter # 리스트나 문자열 안에 있는 빈도수 수집 라이브러리, Counter명사나 단어가 몇번 집계되었는지 라이브러리
import time
options = Options() # 프로토타입객체로 
options.add_argument("--no-sandbox") # 크롬 브라우저 보안정책을 강요하지마라! 안전망 제거
options.add_argument("--disable-dev-shm-usage") # 셀레니움 데이터를 가상의 공간에 넣었다가 다시 찾아오게 하는 옵션
options.add_argument("--headless") # 브라우저창이 나오지 않고 크롤링하도록하는 옵션 "--headless"

driver = webdriver.Chrome(options=options) #크롬이라는 웹드라이버를 사용하는데, 옵션은 위에 설정해놓은 옵션값으로 설정!

url = "https://www.kurly.com/goods/5090270"
test = driver.get(url) #url로 찾아온 값을 test에 넣음
time.sleep(5) #대기시간 주기

#뷰티풀 숲으로 파싱하기
soup = BeautifulSoup(driver.page_source, "html.parser")
#print(soup)
reviews = soup.select("p.css-y49dcn.e36z05c13") #클래스 값으로 찾아오기!
#print(reviews) #p 기준으로 쪼개져있음 
reviews_lists = [review.get_text().strip() for review in reviews] #텍스트만 온전하게 가져오기, 여백제거, reiview
#print(reviews_list)

#찾아왔으니까 나와야함!
driver.quit()

#엑셀저장
wb = Workbook()
ws = wb.active
ws.title = "Kurly Reviews"
ws.append(["리뷰내용"]) #1행에 리뷰내용을 넣기

for item in reviews_lists :
    ws.append([item]) # 두번째 행도 리스트형태로 넣어야 한 줄씩 넣을 수 있음 []

#프로토타입 객체는 실제로 쓸 수 없음. workbook= 변수로만들어서 인스턴트 객체로써 실행되게함. 같은원리로
okt = Okt()
#리스트 형태의 문자열을 하나의 연결된 문자열로 만드는 방법 join
all_text = " ".join(reviews_lists) #여백을 기준으로 reviews_lists의 모든 리스트를 하나의 문자열로 만든다
#print(all_text)
#각각의 명사를 추출, 신조어를 명사로 바꿔주는것 = 정규표현식으로 해결가능! 패턴을 만들어줘서 신조어도 명사로 인식가능!
nouns = okt.nouns(all_text)
filtered_nouns = [n for n in nouns if len(n) > 1] #문자가 2개 이상인 리뷰만 필터시키기
#print(filtered_nouns)

count = Counter(filtered_nouns) #단어가 몇번 나왔는지 카운트로 알려줌!

#새로운 시트를 만들어서 새로운 키워드를 선별한 시트가 있었으면 좋겠음!
ws_keywords = wb.create_sheet("Keyword Frequency") #기존거 놔두고 새로운 시트를 하나 더만들떄
ws_keywords.append(["키워드","빈도 수"]) #첫쨰행 = "키워드",둘째 행 = "빈도 수"] 
for word, freq in count.most_common(30) : #most_common-> 찾아온 카운트의 찾아온 아이템의 개수를 순서대로 나열해주는것
    ws_keywords.append([word, freq])
    

wb.save("Kurly_reviews.xlsx")
print("엑셀저장완료 : Kurly_reviews.xlsx")

엑셀저장완료 : Kurly_reviews.xlsx


In [24]:
from openpyxl import load_workbook # 1, 엑셀 불러오기!
from konlpy.tag import Okt # 4, 한국어를 자연어 처리해줄 수 있는 오픈 라이브러리 / 5, pip install konlpy 설치 우선!
from collections import Counter #14, 각각의 단어의 개수를 확인해서 변수에 넣어보기

wb = load_workbook("연관키워드 20250722 1448.xlsx") # 2, 파일명을 정확하게 복사! wb객체안에 엑셀을 넣음
ws = wb.active # 3, 워크북에서 불러온 엑셀을 활성화

# 6, 우리는 2행 1열부터 데이터가 필요!, 각 행에 값이 있는지 없는지 판단! min_row-> 2행부터 시작한다. max_col_1-> 최대 1열을 넘어가지 않는다.
texts = [row [0] for row in ws.iter_rows(min_row = 2,max_col=1, values_only= True) if row[0]]
# 7, .iter_rows -> 해당 행에 값이 있는지 없는지 여부 판단. values_only= True -> ws안에서 값이 있는지 판단. 값이 있어야지만 유의미하다
#list 형태로 받아오기 위해 for문!, [] 앞뒤에 넣어줌 , row 0번 쨰 값부터 넣을것이다.
#print(texts)

okt = Okt() # 8, okt는 인스턴트 객체가 됨. 자연어이기때문에 
all_text = " ".join(texts) # 9, texts의 값을 여백을 기준으로 한칸씩 떼어서 통문자로 만들자
#print(all_text)

# 10,명사찾기
nouns = okt.nouns(all_text) # 11, nouns 명사를 분리시키는것!
#print(nouns)

#12, 명사의 단어가 1개 이상인 것 중 로 ,데, 안, 은, 같은 쓸데없는것 고르기 위해 명사가 2개이상인 것만 찾아오기
filtered_nouns = [n for n in nouns if len(n)>1] # 13, 명사를 찾아오고 찾아온 문장중에 1 이상인 것만 찾아서 다시 리스트로 만들어라! 
#print(filtered_nouns)

#15, 
count = Counter(filtered_nouns)
#print(count)

#16, 엑셀파일 저장
ws_keywords = wb.create_sheet("키워드 전체분석")# 17, 새로운 시트를 만들때 쓰는 함수
ws_keywords.append(["명사","빈도수"])
# 18, 람다 식(함수)
# 19, sorted 내림차순 정렬, 
# 20, 람다함수 = 익명함수 실행문을 즉각적으로 만들어주는 것. 각각의 키에 맞는 값을 넣어라. 
# 21, reverse = True -> 내림차순 정렬 시키는방법
for word, freq in sorted(count.items(), key = lambda x:x[1], reverse = True) :
    ws_keywords.append([word, freq]) # word 와 freq를 각각의 행에 추가하겠다

# 22, 엑셀 저장
wb.save("연관키워드_분석결과.xlsx")
print("완료 : 전체 키워드 분석 결과 저장됨")


완료 : 전체 키워드 분석 결과 저장됨


In [33]:
#연습해보기 1

from openpyxl import load_workbook
from konlpy.tag import Okt

wb = load_workbook("자외선.xlsx")
ws = wb.active

texts = [row[0] for row in ws.iter_rows(min_row=2,max_col =1, values_only=True) if row[0]]

okt = Okt()
all_text = (" ".join(texts))

nouns = okt.nouns(all_text)
#print(nouns)
filtered_nouns= [n for n in nouns if len(n)>1]
#print(filtered_nouns)
count = Counter(filtered_nouns)

ws_keywords = wb.create_sheet("자외선 분석")
ws_keywords.append(["명사","빈도수"])
for word, freq in sorted(count.items(),key=lambda x:x[1], reverse=True):
    ws_keywords.append([word, freq])

wb.save("자외선 분석결과.xlsx")
print("엑셀저장완료")

엑셀저장완료


In [35]:
#연습해보기 2

from openpyxl import load_workbook
from konlpy.tag import Okt

wb = load_workbook("심리.xlsx")
ws = wb.active

texts = [row[0] for row in ws.iter_rows(min_row=2,max_col =1, values_only=True) if row[0]]

okt = Okt()
all_text = (" ".join(texts))

nouns = okt.nouns(all_text)
filtered_nouns= [n for n in nouns if len(n)>1]
count = Counter(filtered_nouns)
ws_keywords = wb.create_sheet("심리")
ws_keywords.append(["명사","빈도수"])
for word, freq in sorted(count.items(),key=lambda x:x[1], reverse=True):
    ws_keywords.append([word, freq])

wb.save("심리 분석결과.xlsx")
print("엑셀저장완료")

엑셀저장완료


In [36]:
#연습해보기 3

from openpyxl import load_workbook
from konlpy.tag import Okt

wb = load_workbook("상담.xlsx")
ws = wb.active

texts = [row[0] for row in ws.iter_rows(min_row=2,max_col =1, values_only=True) if row[0]]

okt = Okt()
all_text = (" ".join(texts))

nouns = okt.nouns(all_text)
filtered_nouns= [n for n in nouns if len(n)>1]
count = Counter(filtered_nouns)
ws_keywords = wb.create_sheet("상담")
ws_keywords.append(["명사","빈도수"])
for word, freq in sorted(count.items(),key=lambda x:x[1], reverse=True):
    ws_keywords.append([word, freq])

wb.save("상담 분석결과.xlsx")
print("엑셀저장완료")

엑셀저장완료


In [41]:
#한글 + 영어 + 숫자 나오게하는 방법 (정규표현식 쓰기!)
from openpyxl import load_workbook
from konlpy.tag import Okt
from collections import Counter
import re #2, 정규표현식

wb = load_workbook("연관키워드 20250722 1448.xlsx")
ws = wb.active

texts = [row[0] for row in ws.iter_rows(min_row=2,max_col =1, values_only=True) if row[0]]

okt = Okt()
all_text = (" ".join(texts))


#1,tokens 넣기 
nouns_tokens = okt.nouns(all_text)
#3, 정수
regex_tokens = re.findall(r'[A-Za-z]*\d+[A-Za-z]+|\d+[A-Za-z]*',all_text) 
# 4, re.findall() -> ()안에 매칭되어지는 값을 찾아라!
# 5, r'' -> 로우스트링 단순문자열이 아님을 알려주는것!
# 6, * 앞에 영문자들이 있을 수도 있고 없으면 뒤에 것들이 수도 있다! 
# 7, \d -> 10진수의 숫자가들어온다는 것을 알려주는 것!, +는 1개이상있다!

# 8, filtered_nouns= [n for n in nouns if len(n)>1]의 예외의 조건들을 만들어야함.
valid_one_letter = {"금","은","동"} #9, 삭제되면 안되는 단어들!
# 10, 두개의 단어지만 굳이 안써도되는 단어들 -> 직접 여러번 돌리면서 찾아봐야함
stopwords = {"사용","정도", "때문","생각","만족","구매","다음","조금","제품"}
# 11, 두개의 tokens 병합
combined_tokens = nouns_tokens + regex_tokens
# 12, 두글자 이상이거나valid_one_letterdp에 있는 것들 가져와!
filtered_tokens = [t for t in combined_tokens if (len(t)>1 or t in valid_one_letter) and t not in stopwords]
# 13, 불필요한 것들 빼기 not in stopwords and가 나중에 연산되니까 두글자라도 못들어옴! 더 상위에서 걸러짐.

count = Counter(filtered_nouns)

ws_keywords = wb.create_sheet("키워드 전체분석")
ws_keywords.append(["명사","빈도수"])
for word, freq in sorted(count.items(),key=lambda x:x[1], reverse=True):
    ws_keywords.append([word, freq])

wb.save("Kurly_reviews_include_Eng.xlsx")
print("엑셀저장완료")

엑셀저장완료
